# Actions from Trello

This notebook tests pulling data from Trello via the trello APIs.  

It pulls back all actions over a certain period and builds a pandas dataframe with the data for a cumulative flow diagram in Plotly (so we can render in dash ultimately)  

In [ ]:
import requests
import json
import pandas as pd
import numpy as np
import os
from requests_oauthlib import OAuth1Session
import plotly.express as px

Pull in all variables - these are stored as secrets and defined either in the project or passed in to codespaces when it is started. 

In [ ]:
trello_key = os.getenv('TRELLO_KEY')
trello_token = os.getenv('TRELLO_TOKEN')
trello_board_id = os.getenv('TRELLO_BOARD_ID_SHORT')

Run the query against the trello api endpoints. 

In [ ]:
uri = f"https://api.trello.com/1/boards/{trello_board_id}/actions"
filter = "limit=1000&filter=createCard,updateCard"
url = uri + "?" + filter
oauthRequest = OAuth1Session(trello_key,
                    resource_owner_key=trello_token)

headers = {'Accept': "application/json"}
r = oauthRequest.get(url, headers=headers)
print(r.status_code)

In [ ]:
json_data = json.loads(r.text)
len(json_data)

In [ ]:
df_a = pd.json_normalize(json_data)
before_id = df_a["id"].tail(1).values
before_query_id = before_id[0]

In [ ]:
filter = f"limit=1000&filter=createCard,updateCard&before={before_query_id}"
url = uri + "?" + filter
#oauthRequest = OAuth1Session(trello_key,
#                    resource_owner_key=trello_token)

headers = {'Accept': "application/json"}
r = oauthRequest.get(url, headers=headers)
print(r.status_code)

In [ ]:
json_data = json.loads(r.text)
len(json_data)

In [ ]:
df_b = pd.json_normalize(json_data)
before_id = df_b["id"].tail(1).values
before_query_id = before_id[0]

In [ ]:
filter = f"limit=1000&filter=createCard,updateCard&before={before_query_id}"
url = uri + "?" + filter
#oauthRequest = OAuth1Session(trello_key,
#                    resource_owner_key=trello_token)

headers = {'Accept': "application/json"}
r = oauthRequest.get(url, headers=headers)
print(r.status_code)

In [ ]:
json_data = json.loads(r.text)
len(json_data)

In [ ]:
df_c = pd.json_normalize(json_data)
before_id = df_c["id"].tail(1).values
before_query_id = before_id[0]

In [ ]:
frames = [df_a, df_b, df_c]

df = pd.concat(frames)

Extract the results in json

In [ ]:
json_data = json.loads(r.text)

Flatten (normalise) the nested json to the bits that matter

In [ ]:
df = pd.json_normalize(json_data)

Update the date columns to be the index and set the timezone to local. 

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)
df = df.tz_convert("Pacific/Auckland")

Add new blank columns for populating shortly

In [ ]:
df["Count.Landing_Space"] = 0
df["Count.This_Week"] = 0
df["Count.Next"] = 0
df["Count.In_Progress"] = 0
df["Count.Done"] = 0
df["Count.Archive"] = 0

Populate the columns with a 1 if a card moved into the column and -1 as it leaves - to help generate cumulative view 
shortly. 

We also need to tidy up created Cards - This is because the actions don't always attribute  


In [ ]:
# If no list set when created (happens sometimes) set to 'This Week'
df.loc[(df['type'] == 'createCard') & (df['data.list.name'].isnull()),'data.list.name'] = 'This Week'

#Increment for newly created cards
df.loc[(df['type'] == 'createCard') & (df['data.list.name'].str.contains("Landing Space",na=False)),'Count.Landing_Space'] = 1
df.loc[(df['type'] == 'createCard') & (df['data.list.name'].str.contains("This Week",na=False)),'Count.This_Week'] = 1
df.loc[(df['type'] == 'createCard') & (df['data.list.name'].str.contains("In Progress",na=False)),'Count.In_Progress'] = 1
df.loc[(df['type'] == 'createCard') & (df['data.list.name'].str.contains("Next",na=False)),'Count.Next'] = 1
df.loc[(df['type'] == 'createCard') & (df['data.list.name'].str.contains("Done",na=False)),'Count.Done'] = 1
df.loc[(df['type'] == 'createCard') & (df['data.list.name'].str.contains("This Week",na=False)),'Count.This_Week'] = 1

#Increment for moving into a list
df.loc[df['data.listAfter.name'].str.contains("Landing Space",na=False),'Count.Landing_Space'] = 1
df.loc[df['data.listAfter.name'].str.contains("This Week",na=False),'Count.This_Week'] = 1
df.loc[df['data.listAfter.name'].str.contains("In Progress",na=False),'Count.In_Progress'] = 1
df.loc[df['data.listAfter.name'].str.contains("Next",na=False),'Count.Next'] = 1
df.loc[df['data.listAfter.name'].str.contains("Done",na=False),'Count.Done'] = 1

#Decrement for moving out of a list
df.loc[df['data.listBefore.name'].str.contains("Landing Space",na=False),'Count.Landing_Space'] = -1
df.loc[df['data.listBefore.name'].str.contains("This Week",na=False),'Count.This_Week'] = -1
df.loc[df['data.listBefore.name'].str.contains("In Progress",na=False),'Count.In_Progress'] = -1
df.loc[df['data.listBefore.name'].str.contains("Next",na=False),'Count.Next'] = -1
df.loc[df['data.listBefore.name'].str.contains("Done",na=False),'Count.Done'] = -1

#df['Count.Landing_Space'] = df['Count.Landing_Space'].fillna(0)
#df['Count.This_Week'] = df['Count.This_Week'].fillna(0)
#df['Count.In_Progress'] = df['Count.In_Progress'].fillna(0)
#df['Count.Next'] = df['Count.Next'].fillna(0)
#df['Count.Done'] = df['Count.Done'].fillna(0)

Export a subset to csv for testing

In [ ]:
df[["id",
    "data.card.name",
    "type",
    "data.card.closed",
    "data.list.name",
    "data.listBefore.name",
    "data.listAfter.name",
    "appCreator.id",
    "appCreator.name",
    "data.card.idLabels",
    "Count.Landing_Space",
    "Count.This_Week",
    "Count.Next",
    "Count.In_Progress",
    "Count.Done",
    "Count.Archive"
    ]].to_csv("output.csv")

Pull back subset of data that we care about. Rename the columns and unpivot into a [Tidy Data](https://www.jeannicholashould.com/tidy-data-in-python.html) format

In [ ]:
df1 = df[["Count.Landing_Space",
    "Count.This_Week",
    "Count.Next",
    "Count.In_Progress",
    "Count.Done",
    "Count.Archive"
    ]]

df1 = df1.rename(columns={"Count.Landing_Space":"Landing Space", "Count.This_Week":"This Week", "Count.Next":"Next","Count.In_Progress":"In Progress","Count.Done":"Done","Count.Archive":"Archive"})
df1 = (df1
    .reset_index()
    .melt(id_vars='date', var_name='list', value_name='sum')
    #.set_index('date')
)
#formatted_df = formatted_df.sort_values(by=["date"])
#df2.index.name = 'Date'

Group by List and generate cumulative sum per day. 

In [ ]:
df1=df1.sort_values(['date']).reset_index(drop=True)
df1["tickets"]=df1.groupby(['list'])['sum'].cumsum(axis=0)


Create custom sort order for graph rendering

In [ ]:
category_orders_dict = { "list": [  
    "Archive",
    "Done",
    "In Progress",
    "Next",
    "This Week",
    "Landing Space"
                   ]}

Render graph

In [ ]:
fig = px.area(
    df1, x='date', 
    y="tickets", 
    color="list", 
    line_group="list", 
    category_orders=category_orders_dict, 
    title="Cumulative Flow of cards in Personal Kanban",
    labels={ # replaces default labels by column name
                "list": "Trello List",  "date": "Date", "tickets": "# of Cards"
            },
    template="simple_white")
fig.show()

Filter to just show the last 1 most recent action per card based on a specific date. 

TODO: Fix normalsing the start point so that we don't get negative cumulative items. 

TODO: Iterate over generate data for every day


TODO: Render the data in a plotly diagram

TODO: Iterate filling over the data from all time & rendering into a plotly diagram

Troubleshooting: Dump output of json into a pretty nested format. 

In [ ]:
print(json.dumps(json_data, indent = 2, sort_keys=True))

Export to csv for testing

In [ ]:
df.to_csv("output.csv")